# Bayesian Belief Network 

Using `NOTEARS` algorithm for structure learning. 

In [30]:
# import pandas and numpy
import pandas as pd
import numpy as np

# NOTE: codebase adapted from another project, still a WIP
# NEEDS: functional python programming
# silence warnings
import warnings
warnings.filterwarnings("ignore")

# import StructureModel 
from causalnex.structure import StructureModel
sm = StructureModel()


In [31]:
data = pd.read_csv('~/work/phenophasebbn/bbn/rgr_snp_joined.csv')

In [32]:
data.head()

,genotype,SORBI_3004G209300,SORBI_3009G250800,SORBI_3001G305300,SORBI_3003G107500,SORBI_3003G237100,SORBI_3003G242400,SORBI_3002G217300,SORBI_3002G135300,SORBI_3005G126600,...,SORBI_3008G134500,SORBI_3001G214100,SORBI_3002G397600,SORBI_3010G214200,SORBI_3001G358101,SORBI_3001G295800,SORBI_3006G060500,SORBI_3005G179400,season,max_growth_cm_gdd
0,PI144134,0.0,0.0,0.00545,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.005556,0.0,0.005848,0.005457,0.000000,0.0,0.0,0.005405,season_4,0.383763
1,PI144134,0.0,0.0,0.00545,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.005556,0.0,0.005848,0.005457,0.000000,0.0,0.0,0.005405,season_6,0.400702
2,PI145619,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.005405,season_4,0.323791
3,PI145619,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.005405,season_6,0.359087
4,PI145626,0.0,0.0,0.00000,0.005599,0.005618,0.0,0.0,0.0,0.005814,...,0.000000,0.0,0.000000,0.000000,0.005291,0.0,0.0,0.000000,season_4,0.323910


In [43]:
#dummy encode categoricals and create binary vars for sm
from sklearn.preprocessing import LabelEncoder
dum_df = data.copy()


# encode categorical variables as numerical keys
non_numeric_columns = list(dum_df.select_dtypes(exclude=[np.number]).columns)
le = LabelEncoder()
mapping = []
for col in non_numeric_columns:
    dum_df[col] = le.fit_transform(dum_df[col])

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [34]:
#from causalnex.structure.notears import from_pandas
#sm = from_pandas(dum_df, max_iter=1000, h_tol=1e-09, w_threshold=0.95, tabu_edges=None, tabu_parent_nodes=None, tabu_child_nodes=None)

In [79]:
cultivar = [data['genotype'], dum_df['genotype']]

genotypes = pd.concat(cultivar, axis=1)

genotype_uniq = genotypes.drop_duplicates()
genotype_uniq.set_axis(['genotype', 'encoding'], axis=1, inplace=True)


In [80]:
genotype_uniq.head()

,genotype,encoding
0,PI144134,0
2,PI145619,1
4,PI145626,2
6,PI145633,3
8,PI146890,4


In [85]:
genotype_map = dict(zip(genotype_uniq.genotype, genotype_uniq.encoding))
season_map = dict({'season_4': 0, 'season_6': 1})

In [83]:
dum_df.head()

,genotype,SORBI_3004G209300,SORBI_3009G250800,SORBI_3001G305300,SORBI_3003G107500,SORBI_3003G237100,SORBI_3003G242400,SORBI_3002G217300,SORBI_3002G135300,SORBI_3005G126600,...,SORBI_3008G134500,SORBI_3001G214100,SORBI_3002G397600,SORBI_3010G214200,SORBI_3001G358101,SORBI_3001G295800,SORBI_3006G060500,SORBI_3005G179400,season,max_growth_cm_gdd
0,0,0.0,0.0,0.00545,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.005556,0.0,0.005848,0.005457,0.000000,0.0,0.0,0.005405,season_4,0.383763
1,0,0.0,0.0,0.00545,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.005556,0.0,0.005848,0.005457,0.000000,0.0,0.0,0.005405,season_6,0.400702
2,1,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.005405,season_4,0.323791
3,1,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.005405,season_6,0.359087
4,2,0.0,0.0,0.00000,0.005599,0.005618,0.0,0.0,0.0,0.005814,...,0.000000,0.0,0.000000,0.000000,0.005291,0.0,0.0,0.000000,season_4,0.323910


In [84]:
data.head()

,genotype,SORBI_3004G209300,SORBI_3009G250800,SORBI_3001G305300,SORBI_3003G107500,SORBI_3003G237100,SORBI_3003G242400,SORBI_3002G217300,SORBI_3002G135300,SORBI_3005G126600,...,SORBI_3008G134500,SORBI_3001G214100,SORBI_3002G397600,SORBI_3010G214200,SORBI_3001G358101,SORBI_3001G295800,SORBI_3006G060500,SORBI_3005G179400,season,max_growth_cm_gdd
0,PI144134,0.0,0.0,0.00545,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.005556,0.0,0.005848,0.005457,0.000000,0.0,0.0,0.005405,season_4,0.383763
1,PI144134,0.0,0.0,0.00545,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.005556,0.0,0.005848,0.005457,0.000000,0.0,0.0,0.005405,season_6,0.400702
2,PI145619,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.005405,season_4,0.323791
3,PI145619,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.005405,season_6,0.359087
4,PI145626,0.0,0.0,0.00000,0.005599,0.005618,0.0,0.0,0.0,0.005814,...,0.000000,0.0,0.000000,0.000000,0.005291,0.0,0.0,0.000000,season_4,0.323910
